In [ ]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
from shapely.geometry import Point
import numpy as np

In [ ]:
# KONFIGURATION

DB_PATH = 'weihnachten_schnee.db'
GERMANY_GPKG_PATH = 'gadm41_DEU.gpkg'

# Visualisierung
CIRCLE_SIZE = 18
KARTEN_PRO_ZEILE = 10
START_JAHR = 1950
END_JAHR = 2025

# Farbschema
COLOR_SCHNEE = '#f5f5f5'
COLOR_KEIN_SCHNEE = '#1e5f3d'
COLOR_EDGE_SCHNEE = '#b0c4d4'
EDGE_WIDTH = 0.8
ALPHA_CIRCLES = 0.8
COLOR_GERMANY = '#9faab3'  
COLOR_GERMANY_EDGE = '#0c1821'
GERMANY_EDGE_WIDTH = 0.3

In [ ]:
def load_snow_data(db_path, start_jahr, end_jahr):
    """Lädt Schneedaten für alle Jahre"""
    conn = sqlite3.connect(db_path)
    query = f'''
        SELECT d.jahr, s.station_id,
               s.geo_breite as lat, s.geo_laenge as lon,
               MAX(CASE WHEN f.schnee_vorhanden = 1 THEN 1 ELSE 0 END) as hatte_schnee
        FROM fakt_schnee f
        JOIN dim_datum d ON f.datum_id = d.datum_id
        JOIN dim_station s ON f.station_id = s.station_id
        WHERE d.jahr BETWEEN {start_jahr} AND {end_jahr}
        GROUP BY d.jahr, s.station_id, s.geo_breite, s.geo_laenge
    '''
    df = pd.read_sql_query(query, conn)
    conn.close()
    return {jahr: gruppe for jahr, gruppe in df.groupby('jahr')}

def create_map(df_stations, jahr, ax, germany_geom):
    """Erstellt eine Deutschlandkarte für ein Jahr"""
    ax.set_facecolor('none')  
    ax.patch.set_alpha(0)     
    
    # GeoDataFrame erstellen
    geometry = [Point(xy) for xy in zip(df_stations['lon'], df_stations['lat'])]
    gdf = gpd.GeoDataFrame(df_stations, geometry=geometry, crs="EPSG:4326")
    gdf = gdf.to_crs(epsg=25832)
    
    # Deutschland-Grenze plotten
    if germany_geom is not None:
        gdf_germany = gpd.GeoDataFrame([1], geometry=[germany_geom], crs="EPSG:25832")
        gdf_germany.plot(ax=ax, facecolor=COLOR_GERMANY, edgecolor=COLOR_GERMANY_EDGE,
                        linewidth=GERMANY_EDGE_WIDTH, alpha=1.0)
    
    # Stationen plotten
    gdf_kein_schnee = gdf[gdf['hatte_schnee'] == 0]
    gdf_schnee = gdf[gdf['hatte_schnee'] == 1]
    
    if len(gdf_kein_schnee) > 0:
        gdf_kein_schnee.plot(ax=ax, facecolor=COLOR_KEIN_SCHNEE, edgecolor='none',
                            markersize=CIRCLE_SIZE, alpha=ALPHA_CIRCLES)
    
    if len(gdf_schnee) > 0:
        gdf_schnee.plot(ax=ax, facecolor=COLOR_SCHNEE, edgecolor=COLOR_EDGE_SCHNEE,
                       linewidth=EDGE_WIDTH, markersize=CIRCLE_SIZE, alpha=ALPHA_CIRCLES)
    
    # Jahr als Titel
    ax.set_title(f'{jahr}', fontsize=14, fontfamily='Myanmar Text', pad=3, color='white')
    ax.set_axis_off()

def create_visualization(db_path, start_jahr, end_jahr):
    """Erstellt die komplette Visualisierung"""
    # Deutschland-Geometrie laden
    try:
        gdf_germany = gpd.read_file(GERMANY_GPKG_PATH, layer='ADM_ADM_0')
        gdf_germany = gdf_germany.to_crs('EPSG:25832')
        germany_geom = gdf_germany.geometry.union_all()
    except:
        print("⚠ Konnte Deutschland-Geometrie nicht laden")
        germany_geom = None
    
    # Daten laden
    print(f"Lade Daten für {start_jahr}-{end_jahr}...")
    daten_nach_jahr = load_snow_data(db_path, start_jahr, end_jahr)
    jahre = sorted(daten_nach_jahr.keys())
    print(f"✓ {len(jahre)} Jahre geladen\n")
    
    # Grid-Layout berechnen
    n_jahre = len(jahre)
    n_cols = KARTEN_PRO_ZEILE
    n_rows = int(np.ceil(n_jahre / n_cols))
    
    # Figure erstellen mit transparentem Hintergrund
    fig = plt.figure(figsize=(1.5 * n_cols, 2.3 * n_rows + 2)) 
    fig.patch.set_alpha(0)  
    
    # Karten erstellen
    for idx, jahr in enumerate(jahre):
        df_stations = daten_nach_jahr[jahr]
        ax = fig.add_subplot(n_rows, n_cols, idx + 1)
        create_map(df_stations, jahr, ax, germany_geom)
    
    # Layout anpassen 
    plt.subplots_adjust(left=0.01, right=0.99, bottom=0.05, top=0.95,
                       wspace=0.15, hspace=0.1) 
    
    # Speichern mit transparentem Hintergrund
    filename = f"schnee_karten_{start_jahr}_{end_jahr}.png"
    fig.savefig(filename, dpi=400, bbox_inches='tight', 
                transparent=True,  
                pad_inches=0.4)
    print(f"✓ Gespeichert als: {filename}")
    plt.show()
    
    return fig


# HAUPTPROGRAMM
if __name__ == '__main__':
    create_visualization(DB_PATH, START_JAHR, END_JAHR)